In [ ]:
from atmospheric_explorer.cams_interfaces import InversionOptimisedGreenhouseGas, EAC4Instance
from atmospheric_explorer.shapefile import ShapefilesDownloader
from atmospheric_explorer.utils import get_local_folder
from atmospheric_explorer.units_conversion import convert_units_array
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import xarray as xr
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping
import plotly.graph_objects as go
import shutil
import numpy as np
import statsmodels.stats.api as sms
import pandas as pd
from plotly.subplots import make_subplots
from math import ceil
from atmospheric_explorer.plotting_apis import line_with_ci_subplots, clip_and_concat_countries, ghg_surface_satellite_yearly_plot, eac4_anomalies_plot, eac4_hovmoeller_latitude_plot

from atmospheric_explorer.utils import hex_to_rgb
from atmospheric_explorer.data_transformations import confidence_interval

In [ ]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'data'))

In [ ]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'shapefiles'))

In [ ]:
years=[str(y) for y in range(2010, 2021)]
months=[
    '01', '02', '03',
    '04', '05', '06',
    '07', '08', '09',
    '10', '11', '12'
]
countries = ['Italy', 'Russia', 'France', 'Spain', 'Germany', 'Finland']
data_variable = 'carbon_dioxide'
var_name = 'flux_foss'

In [ ]:
fig = ghg_surface_satellite_yearly_plot(data_variable, countries, years, months, "Fossil CO2 flux", var_name)
fig.show()

In [ ]:
data_variable = "total_column_nitrogen_dioxide"
var_name = "tcno2"
countries = ["Italy"]
dates_range = "2018-01-01/2020-01-01"
time_values = "00:00"
title = "Total columns N02 above Italy (anomalies)"

In [ ]:
fig = eac4_anomalies_plot(
    data_variable,
    var_name,
    countries,
    dates_range,
    time_values,
    title,
)
fig.show()

In [ ]:
data_variable = 'total_column_ozone'
var_name = 'gtco3'
dates_range = '2020-01-01/2022-12-31'
time_values = '00:00'
title = 'O3 total column'

In [ ]:
eac4_hovmoeller_latitude_plot(
    data_variable,
    var_name,
    dates_range,
    time_values,
    title,
)